In [1]:
!trtexec

&&&& RUNNING TensorRT.trtexec [TensorRT v8402] # trtexec
=== Model Options ===
  --uff=<file>                UFF model
  --onnx=<file>               ONNX model
  --model=<file>              Caffe model (default = no model, random weights used)
  --deploy=<file>             Caffe prototxt file
  --output=<name>[,<name>]*   Output names (it can be specified multiple times); at least one output is required for UFF and Caffe
  --uffInput=<name>,X,Y,Z     Input blob name and its dimensions (X,Y,Z=C,H,W), it can be specified multiple times; at least one is required for UFF models
  --uffNHWC                   Set if inputs are in the NHWC layout instead of NCHW (use X,Y,Z=H,W,C order in --uffInput)

=== Build Options ===
  --maxBatch                  Set max batch size and build an implicit batch engine (default = same size as --batch)
                              This option should not be used when the input model is ONNX or when dynamic shapes are provided.
  --minShapes=spec            B

In [2]:
import os

In [3]:
ONNX_MODEL_DIR = './Flask/Models/'

FLASK_MODEL_DIR = './Flask/Models/'

TRITON_MODEL_DIR = './Triton/Models/'
TRITON_CONFIG_FILE = 'config.pbtxt'
TRITON_MODEL_FILE = 'model.plan'

In [4]:
def save_tensorrt_model(onnx_model, trt_model, precision, config):
    trt_model = trt_model + '_' + precision
    onnx_model_path = os.path.join(ONNX_MODEL_DIR, onnx_model + '.onnx')
    flask_model_path = os.path.join(FLASK_MODEL_DIR, trt_model + '.plan')

    !trtexec \
        --onnx=$onnx_model_path \
        --explicitBatch \
        --$precision \
        --saveEngine=$flask_model_path

    triton_config_path = os.path.join(TRITON_MODEL_DIR, trt_model, TRITON_CONFIG_FILE)
    os.makedirs(os.path.dirname(triton_config_path), exist_ok=True)
    with open(triton_config_path, 'w') as f:
        f.write(config.strip())

    triton_model_path = os.path.join(TRITON_MODEL_DIR, trt_model, '1', TRITON_MODEL_FILE)
    os.makedirs(os.path.dirname(triton_model_path), exist_ok=True)
    !cp $flask_model_path $triton_model_path
        
    print(flask_model_path)
    print(triton_config_path)
    print(triton_model_path)
    return

In [5]:
onnx_model = 'resnet34-onnx-64'
trt_model = 'resnet34-trt-64'
precision = 'best'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 64, 64 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8402] # trtexec --onnx=./Flask/Models/resnet34-onnx-64.onnx --explicitBatch --best --saveEngine=./Flask/Models/resnet34-trt-64_best.plan
[10/07/2022-07:43:53] [W] --explicitBatch flag has been deprecated and has no effect!
[10/07/2022-07:43:53] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[10/07/2022-07:43:53] [I] === Model Options ===
[10/07/2022-07:43:53] [I] Format: ONNX
[10/07/2022-07:43:53] [I] Model: ./Flask/Models/resnet34-onnx-64.onnx
[10/07/2022-07:43:53] [I] Output:
[10/07/2022-07:43:53] [I] === Build Options ===
[10/07/2022-07:43:53] [I] Max batch: explicit batch
[10/07/2022-07:43:53] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default
[10/07/2022-07:43:53] [I] minTiming: 1
[10/07/2022-07:43:53] [I] avgTiming: 8
[10/07/2022-07:43:53] [I] Precision: FP32+FP16+INT8
[10/07/2022-07:43:53] [I] LayerPrecisi

[10/07/2022-07:44:02] [W] [TRT] Weights [name=Conv_3 + Relu_4.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:02] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:02] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:44:02] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:02] [W] [TRT] Weights [name=Conv_3 + Relu_4.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:02] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:02] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:44:02] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the wei

[10/07/2022-07:44:04] [W] [TRT] Weights [name=Conv_13 + Relu_14.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:04] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:04] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:04] [W] [TRT] Weights [name=Conv_13 + Relu_14.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:04] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:04] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:04] [W] [TRT] Weights [name=Conv_13 + Relu_14.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:04] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:04] [W] [TRT] If this is not the desired behavior, please modify the weights or ret

[10/07/2022-07:44:05] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:05] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:05] [W] [TRT] Weights [name=Conv_21.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:05] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:05] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:06] [W] [TRT] Weights [name=Conv_21.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:06] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:06] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:06] [W] [TRT] Weights [name=Conv_21.weight] had the follow

[10/07/2022-07:44:08] [W] [TRT] Weights [name=Conv_24 + Relu_25.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:08] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:08] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:44:08] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:09] [W] [TRT] Weights [name=Conv_26 + Add_27 + Relu_28.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:09] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:09] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:09] [W] [TRT] Weights [name=Conv_26 + Add_27 + Relu_28.weight] had the following issues when converted to FP16:
[

...

[10/07/2022-07:44:21] [W] [TRT] Weights [name=Conv_72 + Add_74 + Relu_75.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:21] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:21] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:22] [W] [TRT] Weights [name=Conv_76 + Relu_77.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:22] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:22] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:44:22] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:22] [W] [TRT] Weights [name=Conv_76 + Relu_77.weight] had the following issues when converted to FP16:
[10/07/202

[10/07/2022-07:44:25] [W] [TRT] Weights [name=Conv_78 + Add_79 + Relu_80.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:25] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:25] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:44:25] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:25] [W] [TRT] Weights [name=Conv_78 + Add_79 + Relu_80.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:25] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:25] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:44:25] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce th

[10/07/2022-07:44:27] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:27] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:27] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:27] [W] [TRT] Weights [name=Gemm_88.bias] had the following issues when converted to FP16:
[10/07/2022-07:44:27] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:27] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:27] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:27] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:27] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to redu

[10/07/2022-07:44:27] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:27] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:27] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:27] [W] [TRT] Weights [name=Gemm_88.bias] had the following issues when converted to FP16:
[10/07/2022-07:44:27] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:27] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:27] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:27] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:27] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to redu

./Flask/Models/resnet34-trt-64_best.plan
./Triton/Models/resnet34-trt-64_best/config.pbtxt
./Triton/Models/resnet34-trt-64_best/1/model.plan


In [6]:
onnx_model = 'resnet34-onnx-64'
trt_model = 'resnet34-trt-64'
precision = 'fp16'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 64, 64 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8402] # trtexec --onnx=./Flask/Models/resnet34-onnx-64.onnx --explicitBatch --fp16 --saveEngine=./Flask/Models/resnet34-trt-64_fp16.plan
[10/07/2022-07:44:31] [W] --explicitBatch flag has been deprecated and has no effect!
[10/07/2022-07:44:31] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[10/07/2022-07:44:31] [I] === Model Options ===
[10/07/2022-07:44:31] [I] Format: ONNX
[10/07/2022-07:44:31] [I] Model: ./Flask/Models/resnet34-onnx-64.onnx
[10/07/2022-07:44:31] [I] Output:
[10/07/2022-07:44:31] [I] === Build Options ===
[10/07/2022-07:44:31] [I] Max batch: explicit batch
[10/07/2022-07:44:31] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default
[10/07/2022-07:44:31] [I] minTiming: 1
[10/07/2022-07:44:31] [I] avgTiming: 8
[10/07/2022-07:44:31] [I] Precision: FP32+FP16
[10/07/2022-07:44:31] [I] LayerPrecisions: 

[10/07/2022-07:44:36] [W] [TRT] Weights [name=Conv_0 + Relu_1.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:36] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:36] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:44:36] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:36] [W] [TRT] Weights [name=Conv_0 + Relu_1.bias] had the following issues when converted to FP16:
[10/07/2022-07:44:36] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:36] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:36] [W] [TRT] Weights [name=Conv_0 + Relu_1.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:36] [W] [

[10/07/2022-07:44:37] [W] [TRT] Weights [name=Conv_3 + Relu_4.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:37] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:37] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:44:37] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:38] [W] [TRT] Weights [name=Conv_5 + Add_6 + Relu_7.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:38] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:38] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:38] [W] [TRT] Weights [name=Conv_5 + Add_6 + Relu_7.weight] had the following issues when converted to FP16:
[10/07/20

[10/07/2022-07:44:38] [W] [TRT] Weights [name=Conv_13 + Relu_14.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:38] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:38] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:38] [W] [TRT] Weights [name=Conv_15 + Add_16 + Relu_17.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:38] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:38] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:38] [W] [TRT] Weights [name=Conv_15 + Add_16 + Relu_17.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:38] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:38] [W] [TRT] If this is not the desired behavior, please modify 

[10/07/2022-07:44:40] [W] [TRT] Weights [name=Conv_20 + Add_22 + Relu_23.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:40] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:40] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:40] [W] [TRT] Weights [name=Conv_24 + Relu_25.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:40] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:40] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:44:40] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:40] [W] [TRT] Weights [name=Conv_24 + Relu_25.weight] had the following issues when converted to FP16:
[10/07/202

...

[10/07/2022-07:44:49] [W] [TRT] Weights [name=Conv_72 + Add_74 + Relu_75.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:49] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:49] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:50] [W] [TRT] Weights [name=Conv_76 + Relu_77.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:50] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:50] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:44:50] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:50] [W] [TRT] Weights [name=Conv_76 + Relu_77.weight] had the following issues when converted to FP16:
[10/07/202

[10/07/2022-07:44:52] [W] [TRT] Weights [name=Conv_81 + Relu_82.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:52] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:52] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:44:52] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:52] [W] [TRT] Weights [name=Conv_81 + Relu_82.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:52] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:52] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:44:52] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the

[10/07/2022-07:44:53] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:53] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:53] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:53] [W] [TRT] Weights [name=Gemm_88.bias] had the following issues when converted to FP16:
[10/07/2022-07:44:53] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:53] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:53] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:53] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:53] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to redu

[10/07/2022-07:44:54] [W] [TRT] Weights [name=Conv_73.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:54] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:54] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:54] [W] [TRT] Weights [name=Conv_72 + Add_74 + Relu_75.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:54] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:54] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:44:54] [W] [TRT] Weights [name=Conv_76 + Relu_77.weight] had the following issues when converted to FP16:
[10/07/2022-07:44:54] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:44:54] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. C

./Flask/Models/resnet34-trt-64_fp16.plan
./Triton/Models/resnet34-trt-64_fp16/config.pbtxt
./Triton/Models/resnet34-trt-64_fp16/1/model.plan


In [7]:
onnx_model = 'resnet34-onnx-128'
trt_model = 'resnet34-trt-128'
precision = 'fp16'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 128, 128 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""

save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8402] # trtexec --onnx=./Flask/Models/resnet34-onnx-128.onnx --explicitBatch --fp16 --saveEngine=./Flask/Models/resnet34-trt-128_fp16.plan
[10/07/2022-07:47:09] [W] --explicitBatch flag has been deprecated and has no effect!
[10/07/2022-07:47:09] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[10/07/2022-07:47:09] [I] === Model Options ===
[10/07/2022-07:47:09] [I] Format: ONNX
[10/07/2022-07:47:09] [I] Model: ./Flask/Models/resnet34-onnx-128.onnx
[10/07/2022-07:47:09] [I] Output:
[10/07/2022-07:47:09] [I] === Build Options ===
[10/07/2022-07:47:09] [I] Max batch: explicit batch
[10/07/2022-07:47:09] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default
[10/07/2022-07:47:09] [I] minTiming: 1
[10/07/2022-07:47:09] [I] avgTiming: 8
[10/07/2022-07:47:09] [I] Precision: FP32+FP16
[10/07/2022-07:47:09] [I] LayerPrecision

[10/07/2022-07:47:14] [W] [TRT] Weights [name=Conv_0 + Relu_1.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:14] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:14] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:47:14] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:14] [W] [TRT] Weights [name=Conv_0 + Relu_1.bias] had the following issues when converted to FP16:
[10/07/2022-07:47:14] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:14] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:15] [W] [TRT] Weights [name=Conv_3 + Relu_4.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:15] [W] [

[10/07/2022-07:47:16] [W] [TRT] Weights [name=Conv_10 + Add_11 + Relu_12.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:16] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:16] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:16] [W] [TRT] Weights [name=Conv_13 + Relu_14.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:16] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:16] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:16] [W] [TRT] Weights [name=Conv_13 + Relu_14.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:16] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:16] [W] [TRT] If this is not the desired behavior, please modify the weigh

[10/07/2022-07:47:17] [W] [TRT] Weights [name=Conv_21.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:17] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:17] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:18] [W] [TRT] Weights [name=Conv_20 + Add_22 + Relu_23.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:18] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:18] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:18] [W] [TRT] Weights [name=Conv_20 + Add_22 + Relu_23.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:18] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:18] [W] [TRT] If this is not the desired behavior, please modify the weight

[10/07/2022-07:47:19] [W] [TRT] Weights [name=Conv_26 + Add_27 + Relu_28.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:19] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:19] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:19] [W] [TRT] Weights [name=Conv_29 + Relu_30.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:19] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:19] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:19] [W] [TRT] Weights [name=Conv_29 + Relu_30.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:19] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:19] [W] [TRT] If this is not the desired behavior, please modify the weigh

...

[10/07/2022-07:47:30] [W] [TRT] Weights [name=Conv_83 + Add_84 + Relu_85.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:30] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:30] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:30] [W] [TRT] Weights [name=Conv_83 + Add_84 + Relu_85.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:30] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:30] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:30] [W] [TRT] Weights [name=Conv_83 + Add_84 + Relu_85.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:30] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:30] [W] [TRT] If this is not the desired behavior, pleas

[10/07/2022-07:47:31] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:31] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:31] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:31] [W] [TRT] Weights [name=Gemm_88.bias] had the following issues when converted to FP16:
[10/07/2022-07:47:31] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:31] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:31] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:31] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:31] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to redu

[10/07/2022-07:47:31] [W] [TRT] Weights [name=Conv_62 + Add_63 + Relu_64.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:31] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:31] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:31] [W] [TRT] Weights [name=Conv_65 + Relu_66.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:31] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:31] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:47:31] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:31] [W] [TRT] Weights [name=Conv_67 + Add_68 + Relu_69.weight] had the following issues when converted to FP16:
[

[10/07/2022-07:47:35] [I] Average on 10 runs - GPU latency: 0.382812 ms - Host latency: 0.404492 ms (enqueue 0.199585 ms)
[10/07/2022-07:47:35] [I] Average on 10 runs - GPU latency: 0.38269 ms - Host latency: 0.404712 ms (enqueue 0.198486 ms)
[10/07/2022-07:47:35] [I] Average on 10 runs - GPU latency: 0.383057 ms - Host latency: 0.404419 ms (enqueue 0.199292 ms)
[10/07/2022-07:47:35] [I] Average on 10 runs - GPU latency: 0.382593 ms - Host latency: 0.404175 ms (enqueue 0.199365 ms)
[10/07/2022-07:47:35] [I] Average on 10 runs - GPU latency: 0.382227 ms - Host latency: 0.403979 ms (enqueue 0.19917 ms)
[10/07/2022-07:47:35] [I] Average on 10 runs - GPU latency: 0.382544 ms - Host latency: 0.404053 ms (enqueue 0.19939 ms)
[10/07/2022-07:47:35] [I] Average on 10 runs - GPU latency: 0.382349 ms - Host latency: 0.404126 ms (enqueue 0.199268 ms)
[10/07/2022-07:47:35] [I] Average on 10 runs - GPU latency: 0.382642 ms - Host latency: 0.40415 ms (enqueue 0.199194 ms)
[10/07/2022-07:47:35] [I] Av

./Flask/Models/resnet34-trt-128_fp16.plan
./Triton/Models/resnet34-trt-128_fp16/config.pbtxt
./Triton/Models/resnet34-trt-128_fp16/1/model.plan


In [8]:
onnx_model = 'resnet34-onnx-256'
trt_model = 'resnet34-trt-256'
precision = 'fp16'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 256, 256 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8402] # trtexec --onnx=./Flask/Models/resnet34-onnx-256.onnx --explicitBatch --fp16 --saveEngine=./Flask/Models/resnet34-trt-256_fp16.plan
[10/07/2022-07:47:35] [W] --explicitBatch flag has been deprecated and has no effect!
[10/07/2022-07:47:35] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[10/07/2022-07:47:35] [I] === Model Options ===
[10/07/2022-07:47:35] [I] Format: ONNX
[10/07/2022-07:47:35] [I] Model: ./Flask/Models/resnet34-onnx-256.onnx
[10/07/2022-07:47:35] [I] Output:
[10/07/2022-07:47:35] [I] === Build Options ===
[10/07/2022-07:47:35] [I] Max batch: explicit batch
[10/07/2022-07:47:35] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default
[10/07/2022-07:47:35] [I] minTiming: 1
[10/07/2022-07:47:35] [I] avgTiming: 8
[10/07/2022-07:47:35] [I] Precision: FP32+FP16
[10/07/2022-07:47:35] [I] LayerPrecision

[10/07/2022-07:47:40] [W] [TRT] Weights [name=Conv_0 + Relu_1.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:40] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:40] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:47:40] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:40] [W] [TRT] Weights [name=Conv_0 + Relu_1.bias] had the following issues when converted to FP16:
[10/07/2022-07:47:40] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:40] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:40] [W] [TRT] Weights [name=Conv_3 + Relu_4.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:40] [W] [

[10/07/2022-07:47:42] [W] [TRT] Weights [name=Conv_10 + Add_11 + Relu_12.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:42] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:42] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:42] [W] [TRT] Weights [name=Conv_10 + Add_11 + Relu_12.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:42] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:42] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:42] [W] [TRT] Weights [name=Conv_10 + Add_11 + Relu_12.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:42] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:42] [W] [TRT] If this is not the desired behavior, pleas

[10/07/2022-07:47:43] [W] [TRT] Weights [name=Conv_21.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:43] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:43] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:43] [W] [TRT] Weights [name=Conv_21.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:43] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:43] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:43] [W] [TRT] Weights [name=Conv_21.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:43] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:43] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to re

[10/07/2022-07:47:45] [W] [TRT] Weights [name=Conv_24 + Relu_25.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:45] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:45] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:47:45] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:45] [W] [TRT] Weights [name=Conv_26 + Add_27 + Relu_28.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:45] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:45] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:45] [W] [TRT] Weights [name=Conv_26 + Add_27 + Relu_28.weight] had the following issues when converted to FP16:
[

...

[10/07/2022-07:47:54] [W] [TRT] Weights [name=Conv_76 + Relu_77.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:54] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:54] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:47:54] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:54] [W] [TRT] Weights [name=Conv_76 + Relu_77.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:54] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:54] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:47:54] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the

[10/07/2022-07:47:56] [W] [TRT] Weights [name=Conv_81 + Relu_82.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:56] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:56] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:47:56] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:56] [W] [TRT] Weights [name=Conv_83 + Add_84 + Relu_85.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:56] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:56] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:56] [W] [TRT] Weights [name=Conv_83 + Add_84 + Relu_85.weight] had the following issues when converted to FP16:
[

[10/07/2022-07:47:57] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:57] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:57] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:57] [W] [TRT] Weights [name=Gemm_88.bias] had the following issues when converted to FP16:
[10/07/2022-07:47:57] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:57] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:57] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:57] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:57] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to redu

[10/07/2022-07:47:58] [W] [TRT] Weights [name=Conv_70 + Relu_71.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:58] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:58] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:47:58] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:58] [W] [TRT] Weights [name=Conv_73.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:58] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:47:58] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:47:58] [W] [TRT] Weights [name=Conv_72 + Add_74 + Relu_75.weight] had the following issues when converted to FP16:
[10/07/2022-07:47:58

./Flask/Models/resnet34-trt-256_fp16.plan
./Triton/Models/resnet34-trt-256_fp16/config.pbtxt
./Triton/Models/resnet34-trt-256_fp16/1/model.plan


In [9]:
onnx_model = 'resnet34-onnx-512'
trt_model = 'resnet34-trt-512'
precision = 'fp16'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 512, 512 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8402] # trtexec --onnx=./Flask/Models/resnet34-onnx-512.onnx --explicitBatch --fp16 --saveEngine=./Flask/Models/resnet34-trt-512_fp16.plan
[10/07/2022-07:48:02] [W] --explicitBatch flag has been deprecated and has no effect!
[10/07/2022-07:48:02] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[10/07/2022-07:48:02] [I] === Model Options ===
[10/07/2022-07:48:02] [I] Format: ONNX
[10/07/2022-07:48:02] [I] Model: ./Flask/Models/resnet34-onnx-512.onnx
[10/07/2022-07:48:02] [I] Output:
[10/07/2022-07:48:02] [I] === Build Options ===
[10/07/2022-07:48:02] [I] Max batch: explicit batch
[10/07/2022-07:48:02] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default
[10/07/2022-07:48:02] [I] minTiming: 1
[10/07/2022-07:48:02] [I] avgTiming: 8
[10/07/2022-07:48:02] [I] Precision: FP32+FP16
[10/07/2022-07:48:02] [I] LayerPrecision

[10/07/2022-07:48:06] [W] [TRT] Weights [name=Conv_0 + Relu_1.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:06] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:06] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:48:06] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:06] [W] [TRT] Weights [name=Conv_0 + Relu_1.bias] had the following issues when converted to FP16:
[10/07/2022-07:48:06] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:06] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:06] [W] [TRT] Weights [name=Conv_0 + Relu_1.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:06] [W] [

[10/07/2022-07:48:08] [W] [TRT] Weights [name=Conv_5 + Add_6 + Relu_7.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:08] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:08] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:08] [W] [TRT] Weights [name=Conv_5 + Add_6 + Relu_7.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:08] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:08] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:08] [W] [TRT] Weights [name=Conv_5 + Add_6 + Relu_7.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:08] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:08] [W] [TRT] If this is not the desired behavior, please modify 

[10/07/2022-07:48:09] [W] [TRT] Weights [name=Conv_18 + Relu_19.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:09] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:09] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:09] [W] [TRT] Weights [name=Conv_18 + Relu_19.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:09] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:09] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:09] [W] [TRT] Weights [name=Conv_18 + Relu_19.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:09] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:09] [W] [TRT] If this is not the desired behavior, please modify the weights or ret

[10/07/2022-07:48:12] [W] [TRT] Weights [name=Conv_24 + Relu_25.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:12] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:12] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:48:12] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:12] [W] [TRT] Weights [name=Conv_24 + Relu_25.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:12] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:12] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:48:12] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the

...

[10/07/2022-07:48:26] [W] [TRT] Weights [name=Conv_83 + Add_84 + Relu_85.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:26] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:26] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:26] [W] [TRT] Weights [name=Conv_83 + Add_84 + Relu_85.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:26] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:26] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:26] [W] [TRT] Weights [name=Conv_83 + Add_84 + Relu_85.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:26] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:26] [W] [TRT] If this is not the desired behavior, pleas

[10/07/2022-07:48:27] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:27] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:27] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:27] [W] [TRT] Weights [name=Gemm_88.bias] had the following issues when converted to FP16:
[10/07/2022-07:48:27] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:27] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:27] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:27] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:27] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to redu

[10/07/2022-07:48:27] [W] [TRT] Weights [name=Conv_39 + Relu_40.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:27] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:27] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:48:27] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:27] [W] [TRT] Weights [name=Conv_42.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:27] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:27] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:48:27] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.


[10/07/2022-07:48:27] [W] [TRT] Weights [name=Conv_83 + Add_84 + Relu_85.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:27] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:27] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:27] [I] [TRT] Total Host Persistent Memory: 95232
[10/07/2022-07:48:27] [I] [TRT] Total Device Persistent Memory: 25088
[10/07/2022-07:48:27] [I] [TRT] Total Scratch Memory: 0
[10/07/2022-07:48:27] [I] [TRT] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 45 MiB, GPU 2209 MiB
[10/07/2022-07:48:27] [I] [TRT] [BlockAssignment] Algorithm ShiftNTopDown took 1.40147ms to assign 5 blocks to 58 nodes requiring 12582920 bytes.
[10/07/2022-07:48:27] [I] [TRT] Total Activation Memory: 12582920
[10/07/2022-07:48:27] [I] [TRT] [MemUsageChange] Init cuDNN: CPU +0, GPU +10, now: CPU 1754, GPU 1718 (Mi

./Flask/Models/resnet34-trt-512_fp16.plan
./Triton/Models/resnet34-trt-512_fp16/config.pbtxt
./Triton/Models/resnet34-trt-512_fp16/1/model.plan


In [10]:
onnx_model = 'resnet34-onnx-1024'
trt_model = 'resnet34-trt-1024'
precision = 'fp16'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 1024, 1024 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

&&&& RUNNING TensorRT.trtexec [TensorRT v8402] # trtexec --onnx=./Flask/Models/resnet34-onnx-1024.onnx --explicitBatch --fp16 --saveEngine=./Flask/Models/resnet34-trt-1024_fp16.plan
[10/07/2022-07:48:31] [W] --explicitBatch flag has been deprecated and has no effect!
[10/07/2022-07:48:31] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[10/07/2022-07:48:31] [I] === Model Options ===
[10/07/2022-07:48:31] [I] Format: ONNX
[10/07/2022-07:48:31] [I] Model: ./Flask/Models/resnet34-onnx-1024.onnx
[10/07/2022-07:48:31] [I] Output:
[10/07/2022-07:48:31] [I] === Build Options ===
[10/07/2022-07:48:31] [I] Max batch: explicit batch
[10/07/2022-07:48:31] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default
[10/07/2022-07:48:31] [I] minTiming: 1
[10/07/2022-07:48:31] [I] avgTiming: 8
[10/07/2022-07:48:31] [I] Precision: FP32+FP16
[10/07/2022-07:48:31] [I] LayerPrecis

[10/07/2022-07:48:36] [W] [TRT] Weights [name=Conv_0 + Relu_1.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:36] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:36] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:48:36] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:36] [W] [TRT] Weights [name=Conv_0 + Relu_1.bias] had the following issues when converted to FP16:
[10/07/2022-07:48:36] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:36] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:36] [W] [TRT] Weights [name=Conv_0 + Relu_1.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:36] [W] [

[10/07/2022-07:48:39] [W] [TRT] Weights [name=Conv_5 + Add_6 + Relu_7.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:39] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:39] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:39] [W] [TRT] Weights [name=Conv_8 + Relu_9.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:39] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:39] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:48:39] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:39] [W] [TRT] Weights [name=Conv_8 + Relu_9.weight] had the following issues when converted to FP16:
[10/07/2022-07:48

[10/07/2022-07:48:40] [W] [TRT] Weights [name=Conv_18 + Relu_19.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:40] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:40] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:40] [W] [TRT] Weights [name=Conv_18 + Relu_19.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:40] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:40] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:40] [W] [TRT] Weights [name=Conv_18 + Relu_19.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:40] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:40] [W] [TRT] If this is not the desired behavior, please modify the weights or ret

[10/07/2022-07:48:44] [W] [TRT] Weights [name=Conv_24 + Relu_25.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:44] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:44] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:48:44] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:48:45] [W] [TRT] Weights [name=Conv_24 + Relu_25.weight] had the following issues when converted to FP16:
[10/07/2022-07:48:45] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:48:45] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:48:45] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the

...

[10/07/2022-07:49:06] [W] [TRT] Weights [name=Conv_81 + Relu_82.weight] had the following issues when converted to FP16:
[10/07/2022-07:49:06] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:49:06] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:49:06] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:49:06] [W] [TRT] Weights [name=Conv_81 + Relu_82.weight] had the following issues when converted to FP16:
[10/07/2022-07:49:06] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:49:06] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:49:06] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the

[10/07/2022-07:49:08] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/07/2022-07:49:08] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:49:08] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:49:08] [W] [TRT] Weights [name=Gemm_88.bias] had the following issues when converted to FP16:
[10/07/2022-07:49:08] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:49:08] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:49:08] [W] [TRT] Weights [name=Gemm_88.weight] had the following issues when converted to FP16:
[10/07/2022-07:49:08] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:49:08] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to redu

[10/07/2022-07:49:08] [W] [TRT] Weights [name=Conv_55 + Relu_56.weight] had the following issues when converted to FP16:
[10/07/2022-07:49:08] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:49:08] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:49:08] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitude of the weights.
[10/07/2022-07:49:08] [W] [TRT] Weights [name=Conv_57 + Add_58 + Relu_59.weight] had the following issues when converted to FP16:
[10/07/2022-07:49:08] [W] [TRT]  - Subnormal FP16 values detected. 
[10/07/2022-07:49:08] [W] [TRT]  - Values less than smallest positive FP16 Subnormal value detected. Converting to FP16 minimum subnormalized value. 
[10/07/2022-07:49:08] [W] [TRT] If this is not the desired behavior, please modify the weights or retrain with regularization to reduce the magnitu

[10/07/2022-07:49:11] [I] Warmup completed 103 queries over 200 ms
[10/07/2022-07:49:11] [I] Timing trace has 1516 queries over 3.00523 s
[10/07/2022-07:49:11] [I] 
[10/07/2022-07:49:11] [I] === Trace details ===
[10/07/2022-07:49:11] [I] Trace averages of 10 runs:
[10/07/2022-07:49:11] [I] Average on 10 runs - GPU latency: 2.08773 ms - Host latency: 2.69149 ms (enqueue 0.189102 ms)
[10/07/2022-07:49:11] [I] Average on 10 runs - GPU latency: 2.02005 ms - Host latency: 2.61803 ms (enqueue 0.189224 ms)
[10/07/2022-07:49:11] [I] Average on 10 runs - GPU latency: 2.04134 ms - Host latency: 2.65591 ms (enqueue 0.188702 ms)
[10/07/2022-07:49:11] [I] Average on 10 runs - GPU latency: 2.04605 ms - Host latency: 2.64666 ms (enqueue 0.188599 ms)
[10/07/2022-07:49:11] [I] Average on 10 runs - GPU latency: 2.03991 ms - Host latency: 2.64063 ms (enqueue 0.188907 ms)
[10/07/2022-07:49:11] [I] Average on 10 runs - GPU latency: 2.10012 ms - Host latency: 2.70558 ms (enqueue 0.189526 ms)
[10/07/2022-07

./Flask/Models/resnet34-trt-1024_fp16.plan
./Triton/Models/resnet34-trt-1024_fp16/config.pbtxt
./Triton/Models/resnet34-trt-1024_fp16/1/model.plan


In [7]:
onnx_model = 'resnet34-onnx'
trt_model = 'resnet34-trt'
precision = 'int8'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

/bin/sh: 1: trtexec: not found
cp: cannot stat './Flask/Models/resnet34-trt_int8.plan': No such file or directory
./Flask/Models/resnet34-trt_int8.plan
./Triton/Models/resnet34-trt_int8/config.pbtxt
./Triton/Models/resnet34-trt_int8/1/model.plan


In [8]:
onnx_model = 'mobilenet_v2-onnx'
trt_model = 'mobilenet_v2-trt'
precision = 'best'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

/bin/sh: 1: trtexec: not found
cp: cannot stat './Flask/Models/mobilenet_v2-trt_best.plan': No such file or directory
./Flask/Models/mobilenet_v2-trt_best.plan
./Triton/Models/mobilenet_v2-trt_best/config.pbtxt
./Triton/Models/mobilenet_v2-trt_best/1/model.plan


In [9]:
onnx_model = 'mobilenet_v2-onnx'
trt_model = 'mobilenet_v2-trt'
precision = 'fp16'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

/bin/sh: 1: trtexec: not found
cp: cannot stat './Flask/Models/mobilenet_v2-trt_fp16.plan': No such file or directory
./Flask/Models/mobilenet_v2-trt_fp16.plan
./Triton/Models/mobilenet_v2-trt_fp16/config.pbtxt
./Triton/Models/mobilenet_v2-trt_fp16/1/model.plan


In [10]:
onnx_model = 'mobilenet_v2-onnx'
trt_model = 'mobilenet_v2-trt'
precision = 'int8'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

/bin/sh: 1: trtexec: not found
cp: cannot stat './Flask/Models/mobilenet_v2-trt_int8.plan': No such file or directory
./Flask/Models/mobilenet_v2-trt_int8.plan
./Triton/Models/mobilenet_v2-trt_int8/config.pbtxt
./Triton/Models/mobilenet_v2-trt_int8/1/model.plan


In [11]:
onnx_model = 'efficientnet_b0-onnx'
trt_model = 'efficientnet_b0-trt'
precision = 'best'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

/bin/sh: 1: trtexec: not found
cp: cannot stat './Flask/Models/efficientnet_b0-trt_best.plan': No such file or directory
./Flask/Models/efficientnet_b0-trt_best.plan
./Triton/Models/efficientnet_b0-trt_best/config.pbtxt
./Triton/Models/efficientnet_b0-trt_best/1/model.plan


In [12]:
onnx_model = 'efficientnet_b0-onnx'
trt_model = 'efficientnet_b0-trt'
precision = 'fp16'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

/bin/sh: 1: trtexec: not found
cp: cannot stat './Flask/Models/efficientnet_b0-trt_fp16.plan': No such file or directory
./Flask/Models/efficientnet_b0-trt_fp16.plan
./Triton/Models/efficientnet_b0-trt_fp16/config.pbtxt
./Triton/Models/efficientnet_b0-trt_fp16/1/model.plan


In [13]:
onnx_model = 'efficientnet_b0-onnx'
trt_model = 'efficientnet_b0-trt'
precision = 'int8'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

/bin/sh: 1: trtexec: not found
cp: cannot stat './Flask/Models/efficientnet_b0-trt_int8.plan': No such file or directory
./Flask/Models/efficientnet_b0-trt_int8.plan
./Triton/Models/efficientnet_b0-trt_int8/config.pbtxt
./Triton/Models/efficientnet_b0-trt_int8/1/model.plan


In [14]:
onnx_model = 'efficientnet_b7-onnx'
trt_model = 'efficientnet_b7-trt'
precision = 'best'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 600, 600 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

/bin/sh: 1: trtexec: not found
cp: cannot stat './Flask/Models/efficientnet_b7-trt_best.plan': No such file or directory
./Flask/Models/efficientnet_b7-trt_best.plan
./Triton/Models/efficientnet_b7-trt_best/config.pbtxt
./Triton/Models/efficientnet_b7-trt_best/1/model.plan


In [15]:
onnx_model = 'efficientnet_b7-onnx'
trt_model = 'efficientnet_b7-trt'
precision = 'fp16'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 600, 600 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

/bin/sh: 1: trtexec: not found
cp: cannot stat './Flask/Models/efficientnet_b7-trt_fp16.plan': No such file or directory
./Flask/Models/efficientnet_b7-trt_fp16.plan
./Triton/Models/efficientnet_b7-trt_fp16/config.pbtxt
./Triton/Models/efficientnet_b7-trt_fp16/1/model.plan


In [16]:
onnx_model = 'efficientnet_b7-onnx'
trt_model = 'efficientnet_b7-trt'
precision = 'int8'
triton_config = """
platform: "tensorrt_plan"
max_batch_size: 1
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 600, 600 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""
save_tensorrt_model(onnx_model, trt_model, precision, triton_config)

/bin/sh: 1: trtexec: not found
cp: cannot stat './Flask/Models/efficientnet_b7-trt_int8.plan': No such file or directory
./Flask/Models/efficientnet_b7-trt_int8.plan
./Triton/Models/efficientnet_b7-trt_int8/config.pbtxt
./Triton/Models/efficientnet_b7-trt_int8/1/model.plan
